## Setup
Load initial libraries and config to get things up an running.

In [ ]:
import os
import openai
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.vectorstores import Chroma

openai.api_key  = os.environ['OPENAI_API_KEY']

template = """Answer the question based on the context below. Keep the answer short. Use two sentences maxiumum. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}

Question: {question}

Helpful Answer:"""

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

vectordb = Chroma(
    persist_directory="../vectors/",
    embedding_function=OpenAIEmbeddings()
)

question_answerer = ConversationalRetrievalChain.from_llm(
    ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0),
    retriever=vectordb.as_retriever(),
    combine_docs_chain_kwargs={"prompt": PromptTemplate.from_template(template)},
    memory=memory
)

## Chatbot Layout
Setup an interface to interact with the chatbot.

In [ ]:
from IPython.display import HTML
from ipywidgets import widgets

chat_history = []

def text_eventhandler(*args):
    # Needed bc when we "reset" the text input
    # it fires instantly another event since
    # we "changed" it's value to ""
    if args[0]["new"] == "":
        return

    # Show loading animation
    loading_bar.layout.display = "block"

    # Get question
    question = args[0]["new"]

    # Reset text field
    args[0]["owner"].value = ""

    # Formatting question for output
    q = (
        f'<div class="chat-message-right pb-4"><div>'
        + f'<div class="flex-shrink-1 bg-light rounded py-2 px-3 ml-3">'
        + f'<div class="font-weight-bold mb-1">You</div>{question}</div>'
    )

    # Display formatted question
    output.append_display_data(HTML(q))

    try:
        response = question_answerer({"question": f"{question}", "chat_history": chat_history})
        answer = response["answer"]
        chat_history.append((question, answer))
    except Exception as e:
        answer = "<b>Error:</b> " + str(e)

    # Formatting answer for output
    # Replacing all $ otherwise matjax would format them in a strange way
    answer_formatted = answer.replace("$", r"\$")
    a = (
        f'<div class="chat-message-left pb-4"><div>'
        + f'<div class="flex-shrink-1 bg-light rounded py-2 px-3 ml-3">'
        + f'<div class="font-weight-bold mb-1">LLM</div>{answer_formatted}</div>'
    )

    # Turn off loading animation
    loading_bar.layout.display = "none"

    output.append_display_data(HTML(a))

In [ ]:
%%html
<link rel="stylesheet" 
      href="https://cdn.jsdelivr.net/npm/bootstrap@4.5.3/dist/css/bootstrap.min.css" 
      integrity="sha384-TX8t27EcRE3e/ihU7zmQxVncDAy5uIKz4rEkgIXeMed4M0jlfIDPvg6uqKI2xXr2" 
      crossorigin="anonymous">
<style>
    body{margin-top:20px;}

    .chat-message-left,
    .chat-message-right {
        display: flex;
        flex-shrink: 0
    }

    .chat-message-left {
        margin-right: auto
    }

    .chat-message-right {
        flex-direction: row-reverse;
        margin-left: auto
    }
</style>

In [ ]:
from IPython.display import HTML, display

in_text = widgets.Text()
in_text.continuous_update = False
in_text.observe(text_eventhandler, "value")
output = widgets.Output()

file = open("../images/loading.gif", "rb")
image = file.read()
loading_bar = widgets.Image(
    value=image, format="gif", width="20", height="20", layout={"display": "None"}
)

display(
    widgets.HBox(
        [output],
        layout=widgets.Layout(
            width="100%",
            max_height="500px",
            display="inline-flex",
            flex_flow="column-reverse",
        ),
    )
)

display(
    widgets.Box(
        children=[loading_bar, in_text],
        layout=widgets.Layout(display="flex", flex_flow="row"),
    )
)